<a href="https://colab.research.google.com/github/NicolasFerre/ColabFerre/blob/main/Descuentos_Ventas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Herramienta de descuentos con Ventas como el archivo principal

In [26]:
#!pip install python-calamine
#!pip install xlsxwriter

In [27]:
import polars as pl
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font
from openpyxl.utils import column_index_from_string, get_column_letter

In [28]:
loop = True
while loop:
  try:
    dia = input('¿De hace cuántos días es el reporte? ')
    dia = int(dia)
    loop = False
  except:
    print("Ingresa un número válido\n")

¿De hace cuántos días es el reporte? 1


In [29]:
fecha = str(datetime.strftime(datetime.now() - timedelta(dia), "%d-%m-%Y"))

archivo = "Reporte de descuentos " + fecha + ".xlsx"

In [30]:
ofertas = pd.read_excel('Ofertas.xlsx', sheet_name=0, header = 0, engine='calamine')
promos = ofertas[['ean', 'Distr.']]
promos = promos.rename(columns={'ean':'EAN', 'Distr.':'Distribuidor'})

In [31]:
renglonaje = pd.read_excel('Renglonaje.xlsx', engine = 'calamine', sheet_name = 0)
ofertas = pd.read_excel('Ofertas.xlsx', engine = 'calamine')

In [32]:
ventas = pd.read_excel('Ventas.xlsx', engine = 'calamine', header=5)
descuentos = pd.read_excel('Descuentos.xlsx', engine = 'calamine', header=9)

In [33]:
notas = 24

In [34]:
descuentos

,Documento,Unnamed: 1,Fecha,Unnamed: 3,Unnamed: 4,Folio,Unnamed: 6,Cliente,Unnamed: 8,Unnamed: 9,...,Desc %,Unnamed: 20,Unnamed: 21,Descuento $\n,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Total
0,Ticket,NaN,2025-08-18 08:59:02,NaN,NaN,67272.0,NaN,Público en General,NaN,NaN,...,0.00%,NaN,NaN,$ 0.00,NaT,NaN,NaN,NaN,NaN,$ 34.00
1,Factura CFDI,NaN,2025-08-18 09:00:30,NaN,NaN,24191.0,NaN,GRUPO PARLAMENTARIO DEL PARTIDO ACCION NACIONA...,NaN,NaN,...,0.00%,NaN,NaN,$ 0.00,NaT,NaN,NaN,NaN,NaN,$ 198.00
2,Factura CFDI,NaN,2025-08-18 09:02:51,NaN,NaN,24192.0,NaN,GABRIELA CARRETO GUERRERO,NaN,NaN,...,0.00%,NaN,NaN,$ 0.00,NaT,NaN,NaN,NaN,NaN,$ 398.00
3,Factura CFDI,NaN,2025-08-18 09:02:53,NaN,NaN,24193.0,NaN,CARLOS VILLEDA CASIMIRO,NaN,NaN,...,0.00%,NaN,NaN,$ 0.00,NaT,NaN,NaN,NaN,NaN,"$ 3,897.00"
4,Ticket,NaN,2025-08-18 09:03:00,NaN,NaN,67273.0,NaN,Público en General,NaN,NaN,...,0.00%,NaN,NaN,$ 0.00,NaT,NaN,NaN,NaN,NaN,$ 181.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,$ 0.00,NaN,NaT,NaN,NaN,NaN,NaN,NaN
130,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
131,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"$ 66,476.00",NaN,NaT,NaN,NaN,NaN,NaN,NaN
132,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [35]:
descuentos = descuentos[['Folio', 'Desc %', 'Descuento $\n']].dropna()

In [36]:
descuentos

,Folio,Desc %,Descuento $\n
0,67272.0,0.00%,$ 0.00
1,24191.0,0.00%,$ 0.00
2,24192.0,0.00%,$ 0.00
3,24193.0,0.00%,$ 0.00
4,67273.0,0.00%,$ 0.00
...,...,...,...
121,67361.0,0.00%,$ 0.00
122,67362.0,0.00%,$ 0.00
123,67363.0,0.00%,$ 0.00
124,67364.0,0.00%,$ 0.00


In [37]:
df = ventas.merge(descuentos, on='Folio', how='left')

In [38]:
df

,Documento,Unnamed: 1,Unnamed: 2,Fecha,Unnamed: 4,Folio,Unnamed: 6,Cliente,Unnamed: 8,Unnamed: 9,...,Efectivo,Cheque,Crédito,Transferencia,Vales,Tarjeta,Anticipo,SICAR Pagos,Desc %,Descuento $\n
0,Ticket,NaN,NaN,2025-08-18 08:59:02,NaN,67272.0,NaN,Público en General,NaN,NaN,...,$ 34.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,0.00%,$ 0.00
1,PZA,1.0,NaN,NaT,[7506240610960] 12760 Broca para concreto de 1...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Factura CFDI,NaN,NaN,2025-08-18 09:00:30,NaN,24191.0,NaN,GRUPO PARLAMENTARIO DEL PARTIDO ACCION NACIONA...,NaN,NaN,...,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 198.00,$ 0.00,$ 0.00,0.00%,$ 0.00
3,PZA,2.0,NaN,NaT,[7506240669784] 12905 Sellador de poliuretano ...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Factura CFDI,NaN,NaN,2025-08-18 09:02:51,NaN,24192.0,NaN,GABRIELA CARRETO GUERRERO,NaN,NaN,...,$ 398.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,0.00%,$ 0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
447,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
448,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
449,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df = df.dropna(subset=['Documento'])

In [40]:
df = df.loc[(df['Documento'] != 'Notas de Crédito') & (~df['Documento'].str.contains('Página'))]

In [41]:
df['Total   '] = df['Total   '].str.strip().str.replace('$', '').str.replace(',', '').astype(float)

In [42]:
df.to_excel('df.xlsx', index=False)

In [43]:
df = df.rename(columns={'Unnamed: 8': 'EAN', 'Total   ':'Total', 'Unnamed: 9':'Clave',
                        'Unnamed: 10':'Descripción', 'Unnamed: 1':'Cantidad', 'Unnamed: 19':'Precio Unitario',
                        'Unnamed: 27':'Precio', 'Unnamed: 4':'Producto'})

In [44]:
df.insert(1, 'Unidad', '')

In [45]:
df['Unidad'] = df['Documento'].loc[(df['Documento'] != 'Ticket') & (df['Documento'] != 'Factura CFDI')]

In [46]:
df['Documento'] = df['Documento'].loc[(df['Documento'] == 'Ticket') | (df['Documento'] == 'Factura CFDI')]

In [47]:
df['Documento'] = df['Documento'].fillna(0)
df['Folio'] = df['Folio'].fillna(0)
df['Fecha'] = df['Fecha'].fillna(0)
df['Folio'] = df['Folio'].fillna(0)
df['Cliente'] = df['Cliente'].fillna(0)
df['Caja'] = df['Caja'].fillna(0)
df['Usuario'] = df['Usuario'].fillna(0)
df['Est'] = df['Est'].fillna(0)

"""El código generó duplicados por alguna razón (y no en todas las filas), así que los eliminamos"""

df = df.drop_duplicates(df.loc[df['Folio'] != 0], keep='last')

"""Renombramos columnas que sí tienen datos importantes y limpiamos los nulos. Además, añadimos la columna de EAN"""

df['Descripción'] = df['Descripción'].astype(str)

df['Producto'] = df['Producto'].fillna(0)
df['Clave'] = df['Clave'].fillna('----------')
df['Descripción'] = df['Descripción'].fillna(0)
df['Unidad'] = df['Unidad'].fillna(" ")
df['Clave'] = df['Clave'].astype(str)

"""Completamos los datos en las columnas de "Documento", "Fecha", "Cliente", etc."""

documento = df['Documento'][0]
for i in df.index:
  if df['Documento'][i] != 0:
    documento = df['Documento'][i]
  else:
    df.loc[i, 'Documento'] = documento

fecha = df['Fecha'][0]
for i in df.index:
  if df['Fecha'][i] != 0:
    fecha = df['Fecha'][i]
  else:
    df.loc[i, 'Fecha'] = fecha

cliente = df['Cliente'][0]
for i in df.index:
  if df['Cliente'][i] != 0:
    cliente = df['Cliente'][i]
  else:
    df.loc[i, 'Cliente'] = cliente

caja = df['Caja'][0]
for i in df.index:
  if df['Caja'][i] != 0:
    caja = df['Caja'][i]
  else:
    df.loc[i, 'Caja'] = caja

usuario = df['Usuario'][0]
for i in df.index:
  if df['Usuario'][i] != 0:
    usuario = df['Usuario'][i]
  else:
    df.loc[i, 'Usuario'] = usuario

est = df['Est'][0]
for i in df.index:
  if df['Est'][i] != 0:
    est = df['Est'][i]
  else:
    df.loc[i, 'Est'] = est

folio = df['Folio'][0]
for i in df.index:
  if df['Folio'][i] != 0:
    folio = df['Folio'][i]
  else:
    df.loc[i, 'Folio'] = folio

In [48]:
for i in df.index:
  producto = df['Producto'][i]
  if producto != 0:
    a = producto.split(']')
    ean = a[0][1:]
    b = a[1]
    for j in range(len(b)):
      if b[j].isalpha():
        clave = b[1:j]
        descripcion = b[j:]
        break
    df.loc[i, 'EAN'] = ean
    df.loc[i, 'Clave'] = clave.strip()
    df.loc[i, 'Descripción'] = descripcion.strip()

df = df.drop('Producto', axis=1)

/tmp/ipython-input-1375464997.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '7506240669784' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[i, 'EAN'] = ean


In [49]:
df

,Documento,Unidad,Cantidad,Unnamed: 2,Fecha,Folio,Unnamed: 6,Cliente,EAN,Clave,...,Efectivo,Cheque,Crédito,Transferencia,Vales,Tarjeta,Anticipo,SICAR Pagos,Desc %,Descuento $\n
0,Ticket,,NaN,NaN,2025-08-18 08:59:02,67272.0,NaN,Público en General,NaN,----------,...,$ 34.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,0.00%,$ 0.00
2,Factura CFDI,,NaN,NaN,2025-08-18 09:00:30,24191.0,NaN,GRUPO PARLAMENTARIO DEL PARTIDO ACCION NACIONA...,NaN,----------,...,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 198.00,$ 0.00,$ 0.00,0.00%,$ 0.00
3,Factura CFDI,PZA,2.0,NaN,2025-08-18 09:00:30,24191.0,NaN,GRUPO PARLAMENTARIO DEL PARTIDO ACCION NACIONA...,7506240669784,12905,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Factura CFDI,,NaN,NaN,2025-08-18 09:02:51,24192.0,NaN,GABRIELA CARRETO GUERRERO,NaN,----------,...,$ 398.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,0.00%,$ 0.00
5,Factura CFDI,PZA,1.0,NaN,2025-08-18 09:02:51,24192.0,NaN,GABRIELA CARRETO GUERRERO,7506240686170,101571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
429,Ticket,,NaN,NaN,2025-08-18 17:46:05,67364.0,NaN,Público en General,NaN,----------,...,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,"$ 1,250.00",$ 0.00,$ 0.00,0.00%,$ 0.00
430,Ticket,PZA,1.0,NaN,2025-08-18 17:46:05,67364.0,NaN,Público en General,7501206649558,14812,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431,Ticket,PZA,1.0,NaN,2025-08-18 17:46:05,67364.0,NaN,Público en General,7506240641308,15382,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
432,Ticket,,NaN,NaN,2025-08-18 17:47:32,67365.0,NaN,Público en General,NaN,----------,...,$ 9.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,$ 0.00,0.00%,$ 0.00


In [50]:
df['Efectivo'] = df["Efectivo"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Cheque'] = df["Cheque"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Crédito'] = df["Crédito"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Transferencia'] = df["Transferencia"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Vales'] = df["Vales"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Tarjeta'] = df["Tarjeta"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Anticipo'] = df["Anticipo"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['SICAR Pagos'] = df["SICAR Pagos"].str.strip().str.replace('$', '').str.replace(',', '').astype(float)

"""Eliminamos las filas que no tengan un producto asociado"""

# Create a boolean mask for rows where 'Unidad' is one of the desired values
unidades = [' ', 'PZA', 'Set', 'PR', 'XFB', 'XRO', 'XKI', 'KG', 'KGM', 'XBX']
mask = df['Unidad'].isin(unidades)

# Drop rows where the 'Unidad' is NOT in the list of valid units
df = df.drop(df[~mask].index)

"""~mask: This is the inverse of the mask. Since mask is True for rows with valid units, ~mask is True for rows with invalid units.


"""

if df['Precio Unitario'].dtypes == 'object':
  df['Precio Unitario'] = df['Precio Unitario'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df['Precio'] = df['Precio'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
df = df.rename(columns={'Descuento $\n':'Descuento $'})
df['Descuento $'] = df['Descuento $'].str.strip().str.replace('$', '').str.replace(',', '').astype(float)
descuentos = df['Desc %'].str.strip().str.replace('%', '').astype(float)

"""Completamos la columna de "Total"
"""

df['Total'] = df['Total'].fillna(0)
df['Precio'] = df['Precio'].fillna(0)
df['Total'] = df['Total'] + df['Precio']

df['Efectivo'] = df['Efectivo'].fillna(-1)
df['Cheque'] = df['Cheque'].fillna(-1)
df['Crédito'] = df['Crédito'].fillna(-1)
df['Transferencia'] = df['Transferencia'].fillna(-1)
df['Vales'] = df['Vales'].fillna(-1)
df['Tarjeta'] = df['Tarjeta'].fillna(-1)
df['Anticipo'] = df['Anticipo'].fillna(-1)
df['SICAR Pagos'] = df['SICAR Pagos'].fillna(-1)

"""Para evitar el ruido visual, las tendremos columnas vacías en los datos financieros que tengan 0 en su valor total"""

efectivo = df['Efectivo'][0]
for i in df.index:
  if df['Efectivo'][i] != -1:
    if df['Efectivo'][i] == 0:
      efectivo = np.nan
    else:
      efectivo = 1
      df.loc[i, 'Efectivo'] = efectivo
  else:
    df.loc[i, 'Efectivo'] = efectivo

df['Efectivo'] = df['Efectivo'] * df['Total']

cheque = df['Cheque'][0]
for i in df.index:
  if df['Cheque'][i] != -1:
    if df['Cheque'][i] == 0:
      cheque = np.nan
    else:
      cheque = 1
      df.loc[i, 'Cheque'] = cheque
  else:
    df.loc[i, 'Cheque'] = cheque

df['Cheque'] = df['Cheque'] * df['Total']

credito = df['Crédito'][0]
for i in df.index:
  if df['Crédito'][i] != -1:
    if df['Crédito'][i] == 0:
      credito = np.nan
    else:
      credito = 1
      df.loc[i, 'Crédito'] = credito
  else:
    df.loc[i, 'Crédito'] = credito

df['Crédito'] = df['Crédito'] * df['Total']

transferencia = df['Transferencia'][0]
for i in df.index:
  if df['Transferencia'][i] != -1:
    if df['Transferencia'][i] == 0:
      transferencia = np.nan
    else:
      transferencia = 1
      df.loc[i, 'Transferencia'] = transferencia
  else:
    df.loc[i, 'Transferencia'] = transferencia

df['Transferencia'] = df['Transferencia'] * df['Total']

vales = df['Vales'][0]
for i in df.index:
  if df['Vales'][i] != -1:
    if df['Vales'][i] == 0:
      vales = np.nan
    else:
      vales = 1
      df.loc[i, 'Vales'] = vales
  else:
    df.loc[i, 'Vales'] = vales

df['Vales'] = df['Vales'] * df['Total']

tarjeta = df['Tarjeta'][0]
for i in df.index:
  if df['Tarjeta'][i] != -1:
    if df['Tarjeta'][i] == 0:
      tarjeta = np.nan
    else:
      tarjeta = 1
      df.loc[i, 'Tarjeta'] = tarjeta
  else:
    df.loc[i, 'Tarjeta'] = tarjeta

df['Tarjeta'] = df['Tarjeta'] * df['Total']

anticipo = df['Anticipo'][0]
for i in df.index:
  if df['Anticipo'][i] != -1:
    if df['Anticipo'][i] == 0:
      anticipo = np.nan
    else:
      anticipo = 1
      df.loc[i, 'Anticipo'] = anticipo
  else:
    df.loc[i, 'Anticipo'] = anticipo

df['Anticipo'] = df['Anticipo'] * df['Total']

sicar_pagos = df['SICAR Pagos'][0]
for i in df.index:
  if df['SICAR Pagos'][i] != -1:
    if df['SICAR Pagos'][i] == 0:
      sicar_pagos = np.nan
    else:
      sicar_pagos = 1
      df.loc[i, 'SICAR Pagos'] = sicar_pagos
  else:
    df.loc[i, 'SICAR Pagos'] = sicar_pagos

df['SICAR Pagos'] = df['SICAR Pagos'] * df['Total']

"""Completamos la columna de descuentos"""

df['Desc %'] = df['Desc %'].fillna(0)
descuento = df['Desc %'][0]
for i in df.index:
  if df['Desc %'][i] != 0:
    descuento = df['Desc %'][i]
  else:
    df.loc[i, 'Desc %'] = descuento

"""Le damos mejor formato a las columnas"""

df['Unidad'] = df['Unidad'].replace(' ', '----------')
df['Cantidad'] = df['Cantidad'].fillna('----------')
df['Precio Unitario'] = df['Precio Unitario'].fillna('----------')
df['Precio'] = df['Precio'].replace(0, '----------')
df['EAN'] = df['EAN'].replace(0, '----------')
df['Descripción'] = df['Descripción'].replace(0, '----------')

"""Eliminamos las filas sin datos relevantes"""

df = df.drop(df.tail(4).index)

"""Separamos el dato de "Fecha" en Fecha y Hora"""

df.insert(1, "Día", "")
df.insert(2, "Hora", "")

df['Fecha'] = df['Fecha'].astype(str)

for i in df.index:
  fecha = df['Fecha'][i].split(' ')
  df.loc[i, 'Día'] = fecha[0]
  df.loc[i, 'Hora'] = fecha[1]
df.drop('Fecha', axis=1, inplace=True)
df.rename(columns={'Día':'Fecha'}, inplace=True)

df['Descripción'] = df['Descripción'].replace('nan', '----------')

df = df.loc[:, ~df.columns.str.contains('Unnamed')]

"""# Hoja de reporte

Extraemos los datos relevantes del dataframe principal
"""

descuentos = df[['Folio', 'Cliente','Clave', 'EAN', 'Desc %', 'Precio Unitario', 'Cantidad', 'Precio']]

costos = renglonaje[['ean', 'precio mayoreo con IVA', 'precio mínimo de venta', 'precio público con IVA']]
costos = costos.rename(columns={'ean':'EAN'})

costos['EAN'] = costos['EAN'].fillna(0).astype(int)
costos['EAN'] = costos['EAN'].astype(str)

descuentos = descuentos.loc[descuentos['Clave'] != '----------']

promos['EAN'] = promos['EAN'].fillna(0).astype(float).astype(int).astype(str)

descuentos = descuentos.merge(costos, on='EAN', how='left')
descuentos = descuentos.merge(promos, on='EAN', how = 'left')

descuentos['Precio Unitario'] = descuentos['Precio Unitario'].astype(float)
descuentos['precio mínimo de venta'] = descuentos['precio mínimo de venta'].astype(float)
descuentos['precio mayoreo con IVA'] = descuentos['precio mayoreo con IVA'].astype(float)
descuentos['precio público con IVA'] = descuentos['precio público con IVA'].astype(float)

descuentos['Diferencia mayoreo'] = descuentos['Precio Unitario'] - descuentos['precio mayoreo con IVA']
descuentos['Diferencia minimo'] = descuentos['Precio Unitario'] - descuentos['precio mínimo de venta']
descuentos['Diferencia publico'] = descuentos['Precio Unitario'] - descuentos['precio público con IVA']
descuentos['Diferencia dist.'] = descuentos['Precio Unitario'] - descuentos['Distribuidor']

"""Creación del excel"""

writer = pd.ExcelWriter(archivo, engine="xlsxwriter")

df.to_excel(writer, sheet_name='SICAR', index=False)
descuentos.to_excel(writer, sheet_name='Reporte', index=False)

writer.close()

"""# Dando formato al Excel

## Hoja de SICAR
"""

wb = load_workbook(archivo)
ws = wb['SICAR']

"""## Columnas comúnmente usadas"""

last_col = column_index_from_string('P')
col_z = column_index_from_string('Z')
col_dif_mayoreo = column_index_from_string('M')
col_dif_min = column_index_from_string('N')
col_dif_pub = column_index_from_string('O')
col_dif_dist = column_index_from_string('P')

"""### Fuentes
- font_t = Fuente para la fila de metadatos
- font = Fuente para el resto de documentos
"""

font_t = Font(name='Comfortaa', size=12, bold=True, italic=False, color='ffffff')
font = Font(name='Comfortaa', size=11, bold=False, italic=False, color='000000')

"""### Colores de fondo
- fill_b = negro
- fill_r = rojo
- fill_g = verde
- fill_y = amarillo
- fill_p = morado
- fill_bl = azul
"""

fill_b = PatternFill(fill_type="solid", start_color="000000", end_color="000000")
fill_r = PatternFill(fill_type="solid", start_color="ffc7ce", end_color="ffc7ce")
fill_g = PatternFill(fill_type="solid", start_color="b6d7a8", end_color="b6d7a8")
fill_y = PatternFill(fill_type="solid", start_color="ffe599", end_color="ffe599")
fill_p = PatternFill(fill_type="solid", start_color="b4a7d6", end_color="b4a7d6")
fill_bl = PatternFill(fill_type="solid", start_color="a4c2f4", end_color="a4c2f4")

for col in range(1,col_z + 1):
    ws.column_dimensions[get_column_letter(col)].width = 18

ws.column_dimensions['F'].width = 40

for col in range(1, 27):
  ws.cell(1, col).font = font_t
  ws.cell(1, col).alignment = Alignment(horizontal='center')
  ws.cell(1, col).fill = fill_b

for row in range(2, 1000):
  for col in range(1, 27):
    ws.cell(row=row, column=col).font = font
    ws.cell(row=row, column=col).alignment = Alignment(horizontal='center')

for row in range(2, 1000):
  if ws.cell(row=row, column=17).value == None:
    break
  ws.cell(row=row, column=17).fill = fill_g

num_form = '$ #,##0.00'
for row in range(2, 1000):
  for col in range(15, 28):
    ws.cell(row=row, column=col).number_format = num_form

wb.save(archivo)

"""## Hoja de Reporte"""

ws = wb['Reporte']

for col in range(1, last_col + 1):
    ws.column_dimensions[get_column_letter(col)].width = 18

for col in range(1, last_col + 1):
  ws.cell(1, col).font = font_t
  ws.cell(1, col).alignment = Alignment(horizontal='right')
  ws.cell(1, col).fill = fill_b

for row in range(2, 1000):
  for col in range(1, last_col + 1):
    ws.cell(row=row, column=col).font = font
    ws.cell(row=row, column=col).alignment = Alignment(horizontal='right')

"""### Explicación del ciclo for:


Por cada fila desde la 2 hasta la 1000:

---

- SI no hay número de folio, ROMPE CICLO
- SI el valor de diferencia entre el precio y el precio mínimo de venta es nulo, pinta de morado; SI NO -> SI la diferencia con el mayoreo y la diferencia con el mínimo de venta es diferente de 0 -> A

---

A

- SI el valor de descuento es 0.00% -> B; SI NO -> Pinta de amarillo toda la fila

---

B

- SI la diferencia con el precio distribuidor es diferente de 0, pinta de rojo; SI NO, pinta de azul

---
Los colores significan lo siguiente:
---
- El color rojo representa que se dió a un precio menor al mínimo SIN descuento o promoción.

- El color amarillo significa que se dió un precio menor al mínimo CON descuento.

- El color morado representa un error o una fila atípica.

- El azul representa que se vendió un producto con oferta/promoción
"""

for row in range(2, 1000):
  if(ws.cell(row=row, column=1).value == None):
    break

  dif_may = ws.cell(row=row, column=col_dif_mayoreo).value
  dif_min = ws.cell(row=row, column=col_dif_min).value
  dif_pub = ws.cell(row=row, column = col_dif_pub).value
  dif_dist = ws.cell(row=row, column=col_dif_dist).value
  descuento = ws.cell(row=row, column=5).value

  if(dif_min == None):
    for col in range(1, last_col + 1):
      ws.cell(row=row, column=col).fill = fill_p
  elif(dif_may != 0 and dif_min != 0 and dif_pub != 0):
    if(descuento == '0.00%'):
      if(dif_dist != 0):
        for col in range(1, last_col + 1):
          ws.cell(row=row, column=col).fill = fill_r
      else:
        for col in range(1, last_col + 1):
          ws.cell(row=row, column=col).fill = fill_bl
    else:
      for col in range(1, last_col + 1):
        ws.cell(row=row, column=col).fill = fill_y

"""Le da formato de número a las filas de datos financieros"""

num_form = '$ #,##0.00'
for row in range(2, 1000):
  ws.cell(row=row, column=6).number_format = num_form
  for col in range(8, last_col + 1):
    ws.cell(row=row, column=col).number_format = num_form

wb.save(archivo)

resultados = pd.read_excel(archivo, sheet_name=1)
ren = pd.read_excel('Renglonaje.xlsx', sheet_name=2, engine='calamine')
ren = ren.rename(columns={'Total que paga Edy a Truper contando reserva en garantía CON IVA':'Costo', 'código':'Clave'})

ren['Clave'] = ren['Clave'].astype(str)
resultados['Clave'] = resultados['Clave'].astype(str)

ren = ren[['Clave', 'Costo']].dropna()
resultados = resultados.merge(ren, on='Clave', how='left')
resultados['Diferencia costo'] = resultados['Precio Unitario'] - resultados['Costo']

resultados.to_csv('ResultadosD.csv')